本文一共介绍了关于pytorch的api，通过下面五个常用步骤

1. 准备数据（work with data）
2. 构造模型（create model）
3. 优化参数（optimize model parameters）
4. 保存模型（save model）
5. 加载模型（load model）

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# 1. 下载并准备数据集，这里使用datasets自带的FashionMNIST数据集。

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# 2. 定义模型，继承nn.Module

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# 3. 训练模型和测试模型

# 设置损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# 训练循环
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
# 测试模型
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# 运行训练和测试循环
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")      

Epoch 1
-------------------------------
loss: 2.301660  [    0/60000]
loss: 2.283971  [ 6400/60000]
loss: 2.265184  [12800/60000]
loss: 2.260318  [19200/60000]
loss: 2.241578  [25600/60000]
loss: 2.221249  [32000/60000]
loss: 2.222622  [38400/60000]
loss: 2.191046  [44800/60000]
loss: 2.189465  [51200/60000]
loss: 2.159851  [57600/60000]
Test Error: 
 Accuracy: 51.1%, Avg loss: 2.146260 

Epoch 2
-------------------------------
loss: 2.157922  [    0/60000]
loss: 2.143716  [ 6400/60000]
loss: 2.084476  [12800/60000]
loss: 2.109131  [19200/60000]
loss: 2.047733  [25600/60000]
loss: 1.994006  [32000/60000]
loss: 2.024980  [38400/60000]
loss: 1.937651  [44800/60000]
loss: 1.953543  [51200/60000]
loss: 1.885117  [57600/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.871564 

Epoch 3
-------------------------------
loss: 1.905579  [    0/60000]
loss: 1.870024  [ 6400/60000]
loss: 1.754445  [12800/60000]
loss: 1.807166  [19200/60000]
loss: 1.686942  [25600/60000]
loss: 1.642525  [32000/600

In [ ]:
# 4. 保存模型
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
# 5. 加载模型
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# 让tensor在同一个device上运行
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model = model.to(device)   

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Using cuda device
Predicted: "Ankle boot", Actual: "Ankle boot"
